In [2]:
import pandas as pd

# 介護保険事業報告

In [3]:
#要介護者数の作成
file="/Users/yuodanaka/Downloads/use_for_demand_estimate/介護保険事業報告.xls"

#シート2（第1号保険者数）とシート9（第2号保険者数）を読み込み
df1 = pd.read_excel(file, sheet_name=1,index_col=None,skiprows=[0,1,2,3], skipfooter=2)
df2 = pd.read_excel(file, sheet_name=8,index_col=None,skiprows=[0,1,2,3], skipfooter=2)

#それぞれ入らない列の削除
#df1=df1.drop(["Unnamed: 0","合計"],axis=1)
#df2=df2.drop(["Unnamed: 0","合計"],axis=1)

#整形したそれぞれの表をdf_listに格納
df_list=[]
for i in [df1,df2]:
    #入らない列の削除
    tmp=i.drop(["Unnamed: 0","合計"],axis=1)
    #都道府県番号の列を加える
    kn_list=[]
    num_list=list(range(1,48))
    for i in num_list:
        s=str(i)
        s=s.zfill(2)
        kn_list.append(s)
    tmp["都道府県ID"]=kn_list
    df_list.append(tmp)

#df_list内の2つのファイルを結合
df=pd.concat(df_list)

#出力
df.to_csv("/Users/yuodanaka/Downloads/use_for_demand_estimate/care_num_for_demand.csv", index=False)

# 患者調査（患者数）

In [4]:
file1="/Users/yuodanaka/Downloads/use_for_demand_estimate/H27患者調査_傷病別_初診再来.csv"
file2="/Users/yuodanaka/Downloads/use_for_demand_estimate/H27患者調査_傷病別_入院.csv"

#必要な部分だけ読み込み
df1=pd.read_csv(file1, header=None, skiprows=8,skipfooter=120)#初診再来別（（元は総数、病院、診療所とあるが総数のみ）
df2=pd.read_csv(file2, header=None, skiprows=66,usecols=[1])#入院（入院のみ）

##整形（初診再来）
#どのコードでも文字化けするので0列目を消す
df1=df1.drop([0], axis=1)

#必要な傷病の列のみ抽出
df1_new=df1.iloc[[13,14,19,21,23,49]]

#行名をつける
df1_new.index=["糖尿病","精神及び行動の障害","眼及び付属器の疾患","耳及び乳様突起の疾患","高血圧性疾患","妊娠分娩及び産じょく"]

#列から全国の総数を削除
df1_new=df1_new.drop([1,2,3], axis=1)


#初診と再来しか使わないから外来総数は削除
kesu=list(range(4,144,3))
df1_new=df1_new.drop(kesu,axis=1)
df1_new

#列名の変更
name=["初診","再来"]*47
df1_new.columns=name

#初診再来についての情報、繰り返しで都道府県ごとに並んでるから転置する
v_p_df=df1_new.T

#v_p_dfに都道府県番号をつける
kn_list=[]
tmp=list(range(1,48))
for i in tmp:
    s=str(i)
    s=s.zfill(2)
    kn_list.append(s)
    kn_list.append(s)

v_p_df["都道府県ID"]=kn_list


##整形（入院）
#列名を変更
df2.columns=["入院"]

#入らないところを削除（元データには都道府県名のみの行があり、そこでは入院者数が入る列が空白になっている）
df2=df2.dropna()

#該当する傷病の行数を取得
d_l=list(range(14,2779,59))
m_l=list(range(15,2780,59))
ey_l=list(range(20,2785,59))
ea_l=list(range(22,2787,59))
hb_l=list(range(24,2789,59))
p_l=list(range(50,2810,59))

#各リストを足し合わせて、順番に並べる
sick_list=d_l+m_l+ey_l+ea_l+hb_l+p_l
sick_list.sort()

#該当傷病のみの行のみ抽出
df2_new=df2.iloc[sick_list]

#行名を変更
name_list=["糖尿病","精神及び行動の障害","眼及び付属器の疾患","耳及び乳様突起の疾患","高血圧性疾患","妊娠分娩及び産じょく"]*47
df2_new.index=name_list

#v_p_dfと同じスタイルにする
tmp=[]
for i in range(0,282,6):
    j=i+6
    df_tmp=df2_new.iloc[i:j]
    df_tmp=df_tmp.T
    tmp.append(df_tmp)

s_p_df=pd.concat(tmp) 

#s_p_dfに都道府県IDをつける
kn_list=[]
tmp=list(range(1,48))
for i in tmp:
    s=str(i)
    s=s.zfill(2)
    kn_list.append(s)

s_p_df["都道府県ID"]=kn_list


#v_p_dfとs_p_dfを結合
kn_list=[]#都道府県番号のリストを作成
tmp=list(range(1,48))
for i in tmp:
    s=str(i)
    s=s.zfill(2)
    kn_list.append(s)

df_list=[]
for i in kn_list:
    ptmp=v_p_df.query("都道府県ID == @i")
    stmp=s_p_df.query("都道府県ID == @i")
    dftmp=pd.concat([ptmp,stmp])
    df_list.append(dftmp)

p_df=pd.concat(df_list)

#出力
p_df.to_csv("/Users/yuodanaka/Downloads/patient_num_for demand.csv",index=False)

/opt/anaconda3/envs/odanaka/lib/python3.7/site-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """


# 患者調査（平均診療間隔）

In [5]:
#ファイルの読み込み
file3="/Users/yuodanaka/Downloads/use_for_demand_estimate/平均診療間隔.csv"
df3=pd.read_csv(file3, header=None, skiprows=5,skipfooter=120)#必要な行だけ読み込み
df3=df3.drop([0,1],axis=1)#さらに不要な列を削除

#必要な傷病の列のみ抽出
df3_new=df3.iloc[[14,15,20,22,24,50]]

df3_new.index=["糖尿病","精神及び行動の障害","眼及び付属器の疾患","耳及び乳様突起の疾患","高血圧性疾患","妊娠分娩及び産じょく"]

#各傷病が都道府県ごとに繰り返されている
i_p_df=df3_new.T#転置する
#都道府県が「不詳」を消す
i_p_df=i_p_df.drop([49])


#i_p_dfに都道府県IDをつける
kn_list=[]
tmp=list(range(1,48))
for i in tmp:
    s=str(i)
    s=s.zfill(2)
    kn_list.append(s)
i_p_df["都道府県ID"]=kn_list

#出力
i_p_df.to_csv("/Users/yuodanaka/Downloads/use_for_demand_estimate/patient_interval_for_demand.csv",index=False)

/opt/anaconda3/envs/odanaka/lib/python3.7/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


# 国勢調査

In [6]:
#読み込み
file4="/Users/yuodanaka/Downloads/use_for_demand_estimate/国勢調査_年齢性別人口.csv"
df=pd.read_csv(file4,encoding="cp932",skiprows=11)

#不要な行（全国）と列を削除
df=df.iloc[:,1:117]
df=df.drop(["地域コード"],axis=1)

#都道府県・全国だけ抽出
df=df[df["地域識別コード"]=="a"]
#「全国」を削除
df=df[df["Unnamed: 4"]!="全国"]

#総数、男、女をそれぞれ抽出して結合（元データには各分類ごとに国籍で分けたものも入っている）
df1=df[df["※大項目"]==101]
df2=df[df["※大項目"]==201] 
df3=df[df["※大項目"]==301]
df=pd.concat([df1,df2,df3])

#不要になった列を削除
df=df.iloc[:,2:]

#都道府県IDを追加する
id_list=[]
for i in range(1,48):
    s=str(i)
    s=s.zfill(2)
    id_list.append(s)
id_list=id_list*3#総数、男、女で3回繰り返されるため
df["都道府県ID"]=id_list

#列名の変更
c_list=list(df.columns)
c_list[0]="都道府県名"
c_list[1]="total"
for num,i in enumerate(c_list):
    i=i.replace("歳","")
    i=i.replace("以上","〜")
    c_list[num]=i
df.columns=c_list

#df内の「-」を0に変える
df=df.replace("-",0)

#各行で計算して各年齢層ごとの合計
#0〜1	0〜2	6〜12	13〜	10〜50	15〜45	40〜64	65〜	75〜

#0〜1
tmp=df.iloc[:,[2,3]]
tmp=tmp.astype(int)
df["0〜1"]=tmp.sum(axis=1)

#0〜2
tmp=df.iloc[:,[2,3,4]]
tmp=tmp.astype(int)
df["0〜2"]=tmp.sum(axis=1)

#6〜12
tmp=df.iloc[:,8:15]
tmp=tmp.astype(int)
df["6〜12"]=tmp.sum(axis=1)

#13〜
tmp=df.iloc[:,15:113]
tmp=tmp.astype(int)
df["13〜"]=tmp.sum(axis=1)

#10〜50
tmp=df.iloc[:,12:53]
tmp=tmp.astype(int)
df["10〜50"]=tmp.sum(axis=1)

#15〜45
tmp=df.iloc[:,17:48]
tmp=tmp.astype(int)
df["15〜45"]=tmp.sum(axis=1)

#40〜64
tmp=df.iloc[:,42:67]
tmp=tmp.astype(int)
df["40〜64"]=tmp.sum(axis=1)

#65〜
tmp=df.iloc[:,67:113]
tmp=tmp.astype(int)
df["65〜"]=tmp.sum(axis=1)

#75〜
tmp=df.iloc[:,77:113]
tmp=tmp.astype(int)
df["75〜"]=tmp.sum(axis=1)

#出力
df.to_csv("/Users/yuodanaka/Downloads/use_for_demand_estimate/sensus_for_demand.csv",index=False)